# Put Data in AWS DynamoDB

In [ ]:
import pandas as pd
import json 

In [ ]:
js = json.load(open('search_data.json'))
df = pd.read_json(json.dumps(js['summaries']), orient="records")

In [ ]:
# for r in js['summaries'][:100]:
#     print(r)

In [ ]:
# print(df.iloc[73:75])

In [ ]:
df = df.drop([ 'id', 'logo_url', 'popularity'], axis = 1)

In [ ]:
df['architectures'] = df['architectures'].map(lambda x: [i['name'] for i in x if i is not ""])

In [ ]:
df['operating_systems'] = df['operating_systems'].map(lambda x: [i['name'] for i in x if i is not ""])

In [ ]:
df['publisher'] = df['publisher'].map(lambda x: x['name'])

In [ ]:
df['categories'] = df['categories'].map(lambda x: [i['name'] for i in x] if x is not None else "-NA-")

In [ ]:
df = df.fillna({'categories': "None",
           'pull_count': 0})

In [ ]:
df['short_description'] = df['short_description'].map(lambda x: "-NA-" if len(x) <= 1 else x)

In [ ]:
df['categories'] = df['categories'].map(lambda x: '-NA-' if x[0] is '' else x)

In [ ]:
df['operating_systems'] = df['operating_systems'].map(lambda x: x.remove('') if type([]) == type(x) and '' in x else x)

In [ ]:
df['architectures'] = df['architectures'].map(lambda x: x.remove('') if type([]) == type(x) and '' in x else x)

In [ ]:
df['architectures'] = df['architectures'].map(lambda x: '-NA-' if not x else x)

In [ ]:
json.loads(df['architectures'].to_json())

In [ ]:
import boto3
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('docker-data')


In [ ]:
with table.batch_writer() as batch:
    for i, r in df.iterrows():
        batch.put_item(
            Item=json.loads(r.to_json()))
#             print(json.loads(r.to_json()))
#         print(r.to_dict())